In [20]:
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import IterableDataset, Sampler, random_split
import random

### 1.1 Dataset
PyTorch提供了两种数据集定义方式：映射式数据集、可迭代数据集。

#### 1.1.1 映射风格数据集（Map-style datasets）
映射式数据集是实现了 `__getitem__()` 和 `__len__()` 协议的数据集，它表示从索引（可能是非整数） 或键到数据样本的映射。
在加载数据时，PyTorch将自动使用迭代索引(如enumerate)作为key，通过字典索引的方式获取value，本质就是将数据集定义为一个字典。

In [40]:
class KeyMappedDataset(Dataset):
    def __init__(self, data: list):
        self.data = data
    def __getitem__(self, index):
        return self.data[index]
    def __len__(self):
        return len(self.data)

class MySampler(Sampler):
    def __init__(self, datasource):
        super().__init__(datasource)
        self.datasource = datasource
        self.indices = list(range(len(datasource)))  # 获取数据的索引
        random.seed(1024)
        random.shuffle(self.indices)  # 打乱索引顺序
    def __iter__(self):
        # 通过打乱后的索引顺序迭代数据
        for idx in self.indices:
            yield idx
    def __len__(self):
        return len(self.datasource)


_data = ['张三', '李四']
_dataset = KeyMappedDataset(_data)
for item in DataLoader(_dataset, batch_size=2, shuffle=True): # 输出 ['张三', '李四']
    print(item)

_sampler = MySampler(_data)
for item in DataLoader(_dataset, batch_size=2, sampler=_sampler): # 输出 ['李四', '张三']
    print(item)

['张三', '李四']
['李四', '张三']


#### 1.1.2 可迭代数据集（Iterable-style datasets）
迭代器风格。在自定义数据集类中，实现`__iter__`和`__next__`方法，即定义为迭代器。
在后续加载数据迭代时，pytorch将依次获取value，使用这种风格时，需要继承IterableDataset类。这种方法在数据量巨大，无法一下全部加载到内存时非常实用。

In [68]:
class MyIterableDataset(IterableDataset):
    def __init__(self, data):
        super(MyIterableDataset).__init__()
        self.data = data
    def __iter__(self):
        for i in self.data:
            yield i

_data = ['张三', '李四']
_dataset = MyIterableDataset(_data)
for item in DataLoader(_data, batch_size=2): # 输出 tensor([0, 1]) tensor([2, 3])
    print(item)

for item in DataLoader(MyIterableDataset(_data), batch_size=2, shuffle=True):
    print(item)

['张三', '李四']


ValueError: DataLoader with IterableDataset: expected unspecified shuffle option, but got shuffle=True

IterableDataset 在处理大数据集时确实比 Dataset 更有优势，尤其适用于不能一次性加载到内存中的数据集。
然而，IterableDataset 在迭代过程中并没有固定的输出顺序，默认情况下，如果没有实现自定义的打乱机制，样本顺序是按照数据流的顺序输出的。
而且，由于 IterableDataset 并不强制要求实现 __len__() 方法（有时数据总量无法获取），因此不能通过 len() 获取数据集的总量，`不能直接使用 DataLoader 中的 shuffle=True` 来打乱数据。
如果需要打乱数据，需要在 IterableDataset 中实现自定义的打乱机制。

### 1.3 Dataset创建数据集

Dataset创建数据集常用的方法有：
+ 使用 torch.utils.data.TensorDataset 根据Tensor创建数据集(numpy的array，Pandas的DataFrame需要先转换成Tensor)。
+ 使用 torchvision.datasets.ImageFolder 根据图片目录创建图片数据集。<br>
+ 继承 torch.utils.data.Dataset 创建自定义数据集。<br>

此外，还可以通过
+ torch.utils.data.`random_split` 将一个数据集分割成多份，常用于分割训练集，验证集和测试集。<br>
+ 调用Dataset的加法运算符(+)将多个数据集合并成一个数据集。<br>

注意：TensorDataset通过每一个 tensor 的第一个维度进行索引，因此，该类中的 tensor 第一维度必须相等。


In [67]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, random_split, DataLoader

# x, y
_dataset = TensorDataset(torch.tensor(np.random.randint(low=1, high=10, size=(10, 5))), 
                         torch.tensor(np.random.randint(low=1, high=10, size=(10, 1))))

# 查看数据集大小
print(len(_dataset))  # 10

# 划分训练集、测试集
tra_size = int(len(_dataset) * 0.8)
val_size = len(_dataset) - tra_size
generator = torch.Generator().manual_seed(0)

_tra_dataset, _val_dataset = random_split(_dataset, [tra_size, val_size], generator=generator)
print(len(_tra_dataset), len(_val_dataset)) # 8 2

# 按批加载
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_tra_dataloader = DataLoader(_tra_dataset, batch_size=4, shuffle=True)
for batch_idx, (x, y) in enumerate(_tra_dataloader):
    data, labels = x.to(device), y.to(device)  # 将数据和标签移到 GPU
    print(f"Batch {batch_idx + 1}: x device: {data.device}, y device: {labels.device}")

10
8 2
Batch 1: x device: cuda:0, y device: cuda:0
Batch 2: x device: cuda:0, y device: cuda:0
